In [4]:
# load data
parent_dir = "../trace/data/8-12-0.3-400-compressed/"
# parent_dir = "./trace/data/sample/"
data_path = f'{parent_dir}/traffic.data'
index_path = f'{parent_dir}/traffic.index'

In [5]:
# read traffic data index
from ctypes import *
from struct import *
indexes = []
indexes_raw = None
with open(index_path, 'rb') as index_file:
    indexes_raw = index_file.read()
    num_indexes = unpack('Q', indexes_raw[:sizeof(c_uint64())])
    print(num_indexes[0])
    indexes_raw = indexes_raw[sizeof(c_uint64()):]
    for v in iter_unpack('QqQ', indexes_raw):
        indexes.append(v)

400


In [6]:
# read traffic data data
data_raw = open(data_path, 'rb').read()

In [7]:
NUM_PODS = 8
NUM_TORS_PER_POD = 12
NUM_TORS = NUM_PODS * NUM_TORS_PER_POD
# @REFER_TO: networks/jupiter.c:jupiter_get_dataplane
NUM_PAIRS = NUM_TORS ** 2

In [8]:
import numpy as np
from math import sqrt

def get_traffic_matrix(indexes, raw_data, t):
    seek, _, size = [i for i in indexes if i[1] == t][0]
    raw_data = raw_data[seek:seek+size]
    struct_size = 16
    values = [v[0] for v in iter_unpack('f', raw_data[struct_size:])]
    N = int(sqrt(len(values)))
    assert (N*N == len(values))
    return np.array(values).reshape((N, N))

In [9]:
M = get_traffic_matrix(indexes, data_raw, 1)
print(M)

[[   0.    0.    0. ...    0.    0.    0.]
 [   0.    0.    0. ...    0.    0.    0.]
 [   0.    0.    0. ... 5304.    0. 4314.]
 ...
 [ 216.    0.    0. ...    0.    0. 4208.]
 [   0.    0.    0. ...    0.    0.    0.]
 [   0.    0.    0. ...    0.  480.    0.]]


In [16]:
from math import log
maxt = 0
for t in range(400):
    maxt = max(maxt, log(np.max(get_traffic_matrix(indexes, data_raw, t)), 1.3))
    if not t % 20:
        print(maxt)

#TODO: how to simulate such large traffic?

51.158581036905424
57.31056304497754
58.55079185125081
59.07492845454708
60.396292844806176
60.396292844806176
61.260939194965815
61.260939194965815
61.260939194965815
61.260939194965815
61.260939194965815
61.260939194965815
61.260939194965815
61.260939194965815
61.260939194965815
61.260939194965815
61.260939194965815
61.260939194965815
61.260939194965815
61.260939194965815
